# P1 (Соціально-економічний розвиток): 4/4

In [1]:
import pandas as pd
from src.utils import normalize_parameter, save_data
from src.utils import PATH_INTERIM, POPULATION_MAP

---

## p1_01
### сума коштів актуальних (діючих) проєктів міжнародної допомоги, бенефіціаром яких є ОДА, у розрахунку на душу населення
## p1_02
### сума коштів актуальних (діючих) проєктів міжнародної допомоги, бенефіціаром яких є ОДА, у розрахунку на душу населеннясума коштів, залучених за новими проєктами міжнародної допомоги, бенефіціаром яких є ОДА, що почали діяти з початку року до кінця звітного періоду

In [2]:
dict_regions = {'Вінницька': 'Вінницька ОДА',
 'Волинська': 'Волинська ОДА',
 'Дніпропетровська': 'Дніпропетровська ОДА',
 'Донецька': 'Донецька ОДА',
 'Житомирська': 'Житомирська ОДА',
 'Закарпатська': 'Закарпатська ОДА',
 'Запорізька': 'Запорізька ОДА',
 'Івано-Франківська': 'Івано-Франківська ОДА',
 'м. Київ': 'Київська МДА',
 'Київська': 'Київська ОДА',
 'Кіровоградська': 'Кіровоградська ОДА',
 'Луганська': 'Луганська ОДА',
 'Львівська': 'Львівська ОДА',
 'Миколаївська': 'Миколаївська ОДА',
 'Одеська': 'Одеська ОДА',
 'Полтавська': 'Полтавська ОДА',
 'Рівненська': 'Рівненська ОДА',
 'Сумська': 'Сумська ОДА',
 'Тернопільська': 'Тернопільська ОДА',
 'Харківська': 'Харківська ОДА',
 'Херсонська': 'Херсонська ОДА',
 'Хмельницька': 'Хмельницька ОДА',
 'Черкаська': 'Черкаська ОДА',
 'Чернівецька': 'Чернівецька ОДА',
 'Чернігівська': 'Чернігівська ОДА'}

In [3]:
df_P01_001 = pd.read_excel(PATH_INTERIM / "P1" / "P01_001.xlsx", skiprows=2)
df_P01_001["oda_beneficiary"] = df_P01_001["Бенефіціар"].str.contains("\w+\s+ОДА", case=False, na=False).astype(int)
df_P01_001 = df_P01_001.loc[df_P01_001['oda_beneficiary']==1].drop("oda_beneficiary", 1)
df_P01_001["Кінець"] = pd.to_datetime(df_P01_001["Кінець"])
df_P01_001["Початок"] = pd.to_datetime(df_P01_001["Початок"])

In [4]:
# p1_01 актуальні проєкти визначаю як ті, що закінчуться після звітнього періоду

df_P01_001_oda = df_P01_001.loc[
    df_P01_001['Кінець']>=pd.to_datetime('2020-10-01')
].copy()

df_P01_001_rows = []
df_P01_001_columns = ['region','quantity','p1_02_raw','sum_total']
for key, value in dict_regions.items():
    df_P01_001_rows.append(
        [
            key,
            df_P01_001_oda.loc[df_P01_001_oda['Бенефіціар'].str.contains(value)].shape[0], 
            # початок нового звітного періоду - 2020-01-01
            df_P01_001_oda.loc[
                (df_P01_001_oda["Початок"].between("2020-01-01", "2020-10-01")) & 
                (df_P01_001_oda['Бенефіціар'].str.contains(value))]['USD, \nтисяч'].sum()*1000, 
            df_P01_001_oda.loc[df_P01_001_oda['Бенефіціар'].str.contains(value)]['USD, \nтисяч'].sum()*1000
        ]
    )
    
    
df_P01_001_fin = pd.DataFrame(df_P01_001_rows,columns=df_P01_001_columns)
df_P01_001_fin["population"] = df_P01_001_fin["region"].map(POPULATION_MAP)
df_P01_001_fin['p1_01_raw'] = df_P01_001_fin['sum_total'] / df_P01_001_fin['population']
df_P01_001_fin = df_P01_001_fin.loc[df_P01_001_fin["region"].ne("м. Київ")].copy()

In [5]:
df_P01_001_fin["p1_01"] = normalize_parameter(df_P01_001_fin["p1_01_raw"])
df_P01_001_fin["p1_02"] = normalize_parameter(df_P01_001_fin["p1_02_raw"]).fillna(0)

feature_range=(0, 1); fill_na=True; array_bounds=(0.0, 16.208143559586308); normalization_bounds=(0.0, 16.208143559586308)
feature_range=(0, 1); fill_na=True; array_bounds=(0, 0); normalization_bounds=(0, 0)


---

## p1_03
### рівень використання коштів державного фонду регіонального розвитку
## p1_04
### частка коштів, затверджених КМУ на виконання конкретних проєктів ДФРР, відносно загальної суми виділених на область коштів
### (передбачено ДФРР від усіх передбачених коштів)

In [6]:
df_P01_002 = pd.read_excel(PATH_INTERIM / "P1" / "P01_002.xlsx")
df_P01_002 = df_P01_002.loc[df_P01_002["region"].ne("м. Київ")].copy()

# освоєння
df_P01_002["p1_03_raw"] = df_P01_002["Видатки ДФРР"] / df_P01_002["Отримано ДФРР"]
# частка дфрр у фінансуванні проектів
df_P01_002["p1_04_raw"] = df_P01_002["Передбачено ДФРР"] / df_P01_002["Передбачено всього"]


df_P01_002["p1_03"] = normalize_parameter(df_P01_002["p1_03_raw"], min_bound=0, max_bound=1)
df_P01_002["p1_04"] = normalize_parameter(df_P01_002["p1_04_raw"], min_bound=0, max_bound=1)

feature_range=(0, 1); fill_na=True; array_bounds=(0.449978996305826, 0.8646660324524139); normalization_bounds=(0, 1)
feature_range=(0, 1); fill_na=True; array_bounds=(0.46547178001320183, 0.8987824544166384); normalization_bounds=(0, 1)


---

## Група 1: p1_01-p1_02, проєкти міжнародної допомоги
## Група 2: p1_03-p1_04, ДФРР

In [7]:
dict_weights = {
    'p1_01':1,
    'p1_02':1,
    'p1_03':1,
    'p1_04':1,
}

sources = [df_P01_001_fin, df_P01_002]
save_data(sources, dict_weights, 'P1', show_results=True)

,region,p1_01,p1_02,p1_03,p1_04,P1
0,Вінницька,0.000000,0.0,0.726933,0.465472,2.981013
1,Волинська,0.386997,0.0,0.654894,0.898782,4.851683
2,Дніпропетровська,0.389171,0.0,0.653475,0.610856,4.133756
3,Донецька,0.039965,0.0,0.622632,0.726443,3.472601
4,Житомирська,0.017184,0.0,0.667463,0.814445,3.747733
5,Закарпатська,0.000000,0.0,0.699642,0.812536,3.780445
6,Запорізька,0.609396,0.0,0.602185,0.639124,4.626764
7,Івано-Франківська,0.162198,0.0,0.677687,0.768023,4.019772
8,Київська,0.000000,0.0,0.681545,0.546132,3.069190
9,Кіровоградська,0.000000,0.0,0.469955,0.880617,3.376429
